<a href="https://colab.research.google.com/github/BharatSingla12/Play-Store-App-Review-Analysis-EDA/blob/main/Play_Store_App_Review_Analysis_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <b> The Play Store apps data has enormous potential to drive app-making businesses to success. Actionable insights can be drawn for developers to work on and capture the Android market. </b>

## <b> Each app (row) has values for catergory, rating, size, and more. Another dataset contains customer reviews of the android apps.</b>

## <b> Explore and analyze the data to discover key factors responsible for app engagement and success. </b>

## Importing the User Reviews and Play Store Data

## Cleaning the Data

In [ ]:
my_list = ['a', 'b', 'c']
for i in my_list:
  print(i)


a
b
c


In [ ]:
my_list = ['a', 'b', 'c']
for i in my_list:
 print(i)


a
b
c
